In [1]:
from __future__ import division, print_function, absolute_import

import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression

In [2]:
# Data loading and preprocessing
import tflearn.datasets.mnist as mnist
X, Y, testX, testY = mnist.load_data(one_hot=True)
X = X.reshape([-1, 28, 28, 1])
testX = testX.reshape([-1, 28, 28, 1])

Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz


/home/ankdesh/installed/anaconda2/envs/tf0.11/lib/python2.7/gzip.py:275: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  chunk = self.extrabuf[offset: offset + size]
/home/ankdesh/installed/anaconda2/envs/tf0.11/lib/python2.7/site-packages/tflearn/datasets/mnist.py:52: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  data = data.reshape(num_images, rows, cols, 1)


In [3]:
# Building convolutional network
network = input_data(shape=[None, 28, 28, 1], name='input')
network = conv_2d(network, 32, 3, activation='relu', regularizer="L2")
network = max_pool_2d(network, 2)
network = local_response_normalization(network)
network = conv_2d(network, 64, 3, activation='relu', regularizer="L2")
network = max_pool_2d(network, 2)
network = local_response_normalization(network)
network = fully_connected(network, 128, activation='tanh')
network = dropout(network, 0.8)
network = fully_connected(network, 256, activation='tanh')
network = dropout(network, 0.8)
softmax1 = fully_connected(network, 10, activation='softmax')
softmax2 = fully_connected(network, 10, activation='softmax')
network1 = regression(softmax1, optimizer='adam', learning_rate=0.01,
                     loss='categorical_crossentropy', name='target1')
network2 = regression(softmax2, optimizer='adam', learning_rate=0.01,
                     loss='categorical_crossentropy', name='target2')
network = tflearn.merge([network1, network2], mode='elemwise_sum')

In [4]:
model = tflearn.DNN(network, tensorboard_verbose=1)
model.fit({'input': X}, {'target1': Y, 'target2': Y}, n_epoch=2,
           validation_set=({'input': testX}, {'target1': testY, 'target2': testY}),
           snapshot_step=100, show_metric=True, run_id='convnet_mnist')

Training Step: 1720  | total loss: 0.43671
| Adam_0 | epoch: 002 | loss: 0.20250 - acc: 0.9569 | val_loss: 0.12390 - val_acc: 0.9706 -- iter: 55000/55000
| Adam_1 | epoch: 002 | loss: 0.23421 - acc: 0.9481 | val_loss: 0.15234 - val_acc: 0.9579 -- iter: 55000/55000
Training Step: 1720  | total loss: 0.43671
| Adam_0 | epoch: 002 | loss: 0.20250 - acc: 0.9569 | val_loss: 0.12390 - val_acc: 0.9706 -- iter: 55000/55000
| Adam_1 | epoch: 002 | loss: 0.23421 - acc: 0.9481 | val_loss: 0.15234 - val_acc: 0.9579 -- iter: 55000/55000
--
